In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.datasets import mnist
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

print(tf.__version__)

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

2.0.0


Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                             activation='relu',
                             input_shape=input_shape),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("X_train shape: ",x_train.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 104s 2ms/sample - loss: 0.2367 - accuracy: 0.9276 - val_loss: 0.0574 - val_accuracy: 0.9809
Test loss: 0.05736225243178196
Test accuracy: 0.9809
X_train shape:  (60000, 28, 28, 1)


In [5]:
test_image = x_train[0]
test_label = y_train[0]

test_image = np.reshape(test_image, (1,28,28,1))
print("prediction:", model(test_image).numpy().argmax())

prediction: 5


In [6]:
from deep_dream import *

In [8]:
nm = Nightmare(model, test_label, test_label)

In [ ]:
test_imgs = x_train[0:10]
test_labels = y_train[0:10]
for img, label in zip(test_imgs, test_labels):
    tf_img = tf.Variable(img)
    fooled = nm.wrong(tf_img, label, 5)
    faked = nm.fake(tf_img, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 5)
    print("Real Value", model(tf_img).numpy().argmax())
    print("Fooled Value", model(fooled).numpy().argmax())
    print("Faked Value", model(faked).numpy().argmax())